El propósito de este trabajo es entrenar un modelo de ML para intentar predecir la preferencia por la modalidad de trabajo (Física o virtual) en base a las respuestas de los encuestados. Para esto se utilizarán y compararán modelos de Regresión Logística, Random Forest y Support Vector Machine.

# Import de bibliotecas - Carga y preparación de los datos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('Data\efectos_psiquicos_covid_modificado.csv')

In [ ]:
df

,age,gender,occupation,time_bp,time_dp,travel_time,easeof_online,home_env,prod_inc,sleep_bal,new_skill,fam_connect,relaxed,self_time,like_hw,dislike_hw,prefer,certaindays_hw
0,19-25,Male,Student in College,7,5,0.5,3,3,0.0,0.0,0.5,1.0,-0.5,-0.5,100,1,Physical,Yes
1,0-18,Male,Student in School,7,11,0.5,4,2,-0.5,0.5,-1.0,1.0,1.0,1.0,1111,1110,Physical,No
2,19-25,Male,Student in College,7,7,1.5,2,2,1.0,0.0,0.5,0.5,0.5,0.5,1100,111,Physical,Yes
3,19-25,Male,Student in College,7,7,1.5,3,1,0.0,1.0,0.5,0.0,-1.0,-0.5,100,1111,Physical,Yes
4,19-25,Female,Student in College,7,7,1.5,2,2,0.0,0.0,0.0,0.0,0.5,0.0,1010,1000,Physical,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,40-50,Female,Entrepreneur,9,9,0.5,2,1,0.5,-0.5,-0.5,0.5,0.0,0.0,10,1,Physical,Yes
1171,26-32,Female,Homemaker,5,12,0.5,3,5,-0.5,-1.0,-0.5,0.0,-0.5,-0.5,100,111,Physical,Maybe
1172,26-32,Male,Working Professional,9,11,0.5,3,2,0.5,0.5,0.5,0.5,0.0,0.0,1111,110,Physical,Maybe
1173,26-32,Male,Working Professional,11,12,1.5,2,2,0.0,0.0,0.0,-1.0,0.0,-0.5,1111,1100,Physical,Yes


# Modelado

In [ ]:
# Normalizar variables categóricas realizando un mapeo con sus frecuencias relativas
data = df
for i in ['age','gender','occupation','certaindays_hw']:
    data[i] = data[i].map(data[i].value_counts(normalize=True))
data

,age,gender,occupation,time_bp,time_dp,travel_time,easeof_online,home_env,prod_inc,sleep_bal,new_skill,fam_connect,relaxed,self_time,like_hw,dislike_hw,prefer,certaindays_hw
0,0.293617,0.552340,0.304681,7,5,0.5,3,3,0.0,0.0,0.5,1.0,-0.5,-0.5,100,1,Physical,0.483404
1,0.062979,0.552340,0.015319,7,11,0.5,4,2,-0.5,0.5,-1.0,1.0,1.0,1.0,1111,1110,Physical,0.262979
2,0.293617,0.552340,0.304681,7,7,1.5,2,2,1.0,0.0,0.5,0.5,0.5,0.5,1100,111,Physical,0.483404
3,0.293617,0.552340,0.304681,7,7,1.5,3,1,0.0,1.0,0.5,0.0,-1.0,-0.5,100,1111,Physical,0.483404
4,0.293617,0.440851,0.304681,7,7,1.5,2,2,0.0,0.0,0.0,0.0,0.5,0.0,1010,1000,Physical,0.483404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,0.154043,0.440851,0.101277,9,9,0.5,2,1,0.5,-0.5,-0.5,0.5,0.0,0.0,10,1,Physical,0.483404
1171,0.222128,0.440851,0.069787,5,12,0.5,3,5,-0.5,-1.0,-0.5,0.0,-0.5,-0.5,100,111,Physical,0.253617
1172,0.222128,0.552340,0.407660,9,11,0.5,3,2,0.5,0.5,0.5,0.5,0.0,0.0,1111,110,Physical,0.253617
1173,0.222128,0.552340,0.407660,11,12,1.5,2,2,0.0,0.0,0.0,-1.0,0.0,-0.5,1111,1100,Physical,0.483404


In [ ]:
# Separar los features y el target
X = data.drop(columns='prefer', axis=1)
y = data["prefer"]

In [ ]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Escalar los datos con MinMaxScaler
scaler = MinMaxScaler()
X_train[['time_bp', 'time_dp','easeof_online', 'home_env','like_hw', 'dislike_hw']] = scaler.fit_transform(X_train[['time_bp', 'time_dp','easeof_online', 'home_env','like_hw', 'dislike_hw']])
X_test[['time_bp', 'time_dp','easeof_online', 'home_env','like_hw', 'dislike_hw']] = scaler.transform(X_test[['time_bp', 'time_dp','easeof_online', 'home_env','like_hw', 'dislike_hw']])

In [ ]:
X_train.shape, X_test.shape

((940, 17), (235, 17))

In [ ]:
X_train

,age,gender,occupation,time_bp,time_dp,travel_time,easeof_online,home_env,prod_inc,sleep_bal,new_skill,fam_connect,relaxed,self_time,like_hw,dislike_hw,certaindays_hw
135,0.293617,0.552340,0.304681,0.125,0.000,0.5,0.00,0.50,0.0,0.0,1.0,1.0,0.5,1.0,0.999099,0.900000,0.483404
796,0.222128,0.552340,0.407660,0.625,0.875,0.5,0.00,1.00,1.0,-1.0,0.5,0.5,0.0,0.0,1.000000,1.000000,0.483404
247,0.293617,0.552340,0.304681,0.625,0.125,2.5,0.25,0.75,0.0,-0.5,1.0,1.0,0.5,1.0,0.990090,1.000000,0.483404
643,0.144681,0.440851,0.069787,0.125,1.000,0.5,0.00,0.25,1.0,-0.5,-0.5,-0.5,-0.5,-0.5,0.098198,0.000000,0.253617
184,0.293617,0.552340,0.304681,0.375,0.125,0.5,0.25,0.00,0.5,0.5,1.0,1.0,0.5,1.0,0.900000,0.900000,0.483404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0.086809,0.552340,0.407660,0.625,0.875,0.5,0.25,0.50,0.0,0.5,0.5,0.0,0.5,0.0,0.900000,0.089189,0.262979
1095,0.222128,0.440851,0.069787,0.125,1.000,0.5,0.50,1.00,-0.5,-1.0,-0.5,0.0,-0.5,-0.5,0.089189,0.099099,0.253617
1130,0.154043,0.440851,0.101277,0.625,0.625,0.5,0.25,0.00,0.5,-0.5,-0.5,0.5,0.0,0.0,0.008108,0.000000,0.483404
860,0.144681,0.440851,0.407660,0.375,0.375,2.5,1.00,0.75,-0.5,0.5,0.5,1.0,0.5,0.5,0.900901,0.909009,0.262979


In [ ]:
X_test

,age,gender,occupation,time_bp,time_dp,travel_time,easeof_online,home_env,prod_inc,sleep_bal,new_skill,fam_connect,relaxed,self_time,like_hw,dislike_hw,certaindays_hw
661,0.154043,0.440851,0.062128,1.000,1.000,1.5,0.50,0.25,-0.5,-0.5,-1.0,-1.0,-1.0,-0.5,0.990090,1.000000,0.262979
427,0.222128,0.440851,0.407660,0.625,0.375,0.5,0.00,0.00,1.0,1.0,1.0,1.0,1.0,1.0,0.990090,0.900000,0.483404
192,0.293617,0.552340,0.304681,0.375,0.000,1.5,0.00,0.00,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,0.089189,0.483404
209,0.293617,0.552340,0.304681,0.375,0.000,0.5,0.50,0.75,-0.5,-1.0,0.5,0.5,-0.5,0.5,0.999099,1.000000,0.262979
738,0.062979,0.552340,0.304681,0.375,0.000,0.5,0.75,1.00,-0.5,0.0,0.0,0.5,0.0,0.5,0.089189,0.000000,0.483404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,0.154043,0.440851,0.407660,0.625,0.875,0.5,0.50,0.50,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.900000,0.483404
43,0.293617,0.552340,0.304681,0.125,0.375,0.5,0.50,1.00,0.5,-0.5,0.0,0.5,0.0,0.0,0.098198,1.000000,0.253617
486,0.154043,0.440851,0.407660,0.625,0.625,2.5,0.25,0.25,0.0,0.0,0.5,0.5,0.5,0.5,0.999099,0.099099,0.483404
1174,0.154043,0.552340,0.407660,0.125,0.125,0.5,0.25,0.25,0.5,-0.5,0.5,0.5,0.5,0.5,0.990090,0.090090,0.483404


## Regresión Logística

In [ ]:
# Inicializar el modelo
rl = LogisticRegression(random_state=42)

# Entrenar el modelo
rl.fit(X_train, y_train)

# Evaluar el modelo
y_pred = rl.predict(X_test)

print( 'Accuracy Score',accuracy_score(y_test, y_pred))
print("\nClassification Report: \n", classification_report(y_test, y_pred))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

Accuracy Score 0.902127659574468

Classification Report: 
               precision    recall  f1-score   support

    Physical       0.91      0.96      0.93       172
      Remote       0.87      0.75      0.80        63

    accuracy                           0.90       235
   macro avg       0.89      0.85      0.87       235
weighted avg       0.90      0.90      0.90       235

Confusion Matrix: 
 [[165   7]
 [ 16  47]]


El modelo de regresión logística tiene una exactitud del 90.21% en el conjunto de prueba. La precisión, el recall y la puntuación F1 también son bastante buenas, lo que sugiere que el modelo está funcionando bien al identificar ambas clases.

Precisión: La capacidad del clasificador para no etiquetar una muestra positiva como negativa. En nuestro caso, para la clase 'Physical' la precisión es del 0.91, mientras que para la clase 'Remote', la precisión es del 0.87. Esto significa que el modelo es bueno para evitar falsos positivos.

Recall: La capacidad del clasificador para encontrar todas las muestras positivas. La recuperación para la clase 'Physical' es del 0.96, y para la 'Remote' es del 0.76. Esto significa que el modelo es bastante mejor para encontrar individuos que prefieren trabajar de manera presencial en comparación con aquellos que prefieren el trabajo remoto.

Puntuación F1: La media armónica ponderada de la precisión y la recuperación (recall). La puntuación F1 alcanza su mejor valor en 'Physical' y un valor algo más bajo en 'Remote'. La puntuación F1 para 'Physical' es del 0.93, y para 'Remote' es del 0.80. Esto indica un rendimiento dentro de todo equilibrado del modelo en ambas clases, pero con un mejor rendimiento para 'Physical'.

## Random Forest

In [ ]:
# Definir los hiperparámetros a buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Inicializar el clasificador RandomForest
rf_clf = RandomForestClassifier(random_state=42)

# Inicializar GridSearchCV
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Realizar la búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo encontrado
best_rf_model = grid_search.best_estimator_

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el mejor modelo
y_pred = best_rf_model.predict(X_test)

print( 'Accuracy Score', accuracy_score(y_test, y_pred))
print("\nClassification Report: \n", classification_report(y_test, y_pred))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

Mejores hiperparámetros encontrados:
{'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Accuracy Score 0.9276595744680851

Classification Report: 
               precision    recall  f1-score   support

    Physical       0.92      0.98      0.95       172
      Remote       0.94      0.78      0.85        63

    accuracy                           0.93       235
   macro avg       0.93      0.88      0.90       235
weighted avg       0.93      0.93      0.93       235

Confusion Matrix: 
 [[169   3]
 [ 14  49]]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


El modelo de Random Forest tiene una exactitud del 92.76% en el conjunto de prueba, una poco mejor que el Modelo de Regresión Logística. La precisión, la recuperación (recall) y la puntuación F1 también son bastante buenas, lo que sugiere que el modelo está funcionando bien al identificar ambas clases.

Precisión: La capacidad del clasificador para no etiquetar una muestra positiva como negativa. En este caso, la presisión para ambas clases es bastante similar, con una leve ventaja para la clase 'Remote' con un 0.94, contra un 0.92 de la clase 'Physical'. Esto significa que el modelo es muy bueno para evitar falsos positivos.

Recall: La capacidad del clasificador para encontrar todas las muestras positivas. La recuperación para 'Physical' es del 0.98, y para 'Remote' es del 0.78. Al igual que la Regresión Logística, este modelo es bastante mejor para encontrar individuos que prefieren trabajar de manera presencial en comparación con aquellos que prefieren el trabajo remoto, aunque con una leve mejoría.

Puntuación F1: La media armónica ponderada de precisión y recuperación (recall). Al igual que en la Regresión Logística, la puntuación F1 alcanza su mejor valor en 'Physical' y un valor algo más bajo en 'Remote', pero con una leve mejoría. La puntuación F1 para 'Physical' es del 0.95, y para 'Remote' es del 0.85. Esto indica un rendimiento dentro de todo equilibrado del modelo en ambas clases, pero con un mejor rendimiento para 'Physical'.

## Support Vector Machine

In [ ]:
# Definir los hiperparámetros a buscar
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'coef0': [0.0, 0.1, 0.5, 1.0]
}

# Inicializar el clasificador SVC
svm_clf = SVC(random_state=42)

# Inicializar GridSearchCV
grid_search = GridSearchCV(estimator=svm_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Realizar la búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo encontrado
best_svm_model = grid_search.best_estimator_

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el mejor modelo
y_pred = best_svm_model.predict(X_test)

print( 'Accuracy Score', accuracy_score(y_test, y_pred))
print("\nClassification Report: \n", classification_report(y_test, y_pred))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

Mejores hiperparámetros encontrados:
{'C': 1, 'coef0': 1.0, 'degree': 4, 'gamma': 0.1, 'kernel': 'poly'}
Accuracy Score 0.9276595744680851

Classification Report: 
               precision    recall  f1-score   support

    Physical       0.94      0.96      0.95       172
      Remote       0.88      0.84      0.86        63

    accuracy                           0.93       235
   macro avg       0.91      0.90      0.91       235
weighted avg       0.93      0.93      0.93       235

Confusion Matrix: 
 [[165   7]
 [ 10  53]]


Al igual que el modelo de Random Forest, el modelo SVM tiene una exactitud del 92.76% en el conjunto de prueba, un poco más que el Modelo de Regresión Logística. La precisión, la recuperación (recall) y la puntuación F1 también son bastante buenas, lo que sugiere que el modelo está funcionando bien al identificar ambas clases.

Precisión: La capacidad del clasificador para no etiquetar una muestra positiva como negativa. A diferencia del Random Forest, en este modelo la diferencia de precisión entre ambas clases es mayor. Igualmente, este modelo es muy bueno para evitar falsos positivos, principalmente en la clase 'Physical' con una precisión de 0.94, y un 0.88 para la clase 'Remote'.

Recall: La capacidad del clasificador para encontrar todas las muestras positivas. La recuperación para 'Physical' es del 0.96, y para 'Remote' es del 0.84. Al igual que los modelos anteriores, este modelo es bastante mejor para encontrar individuos que prefieren trabajar de manera presencial en comparación con aquellos que prefieren el trabajo remoto, aunque con una leve mejoría para la clase 'Remote' achicando la brecha entre ambas clases.

Puntuación F1: La media armónica ponderada de precisión y recuperación (recall). Al igual que en los modelos anteriores, la puntuación F1 alcanza su mejor valor en 'Physical' y un valor algo más bajo en 'Remote', siendo prácticamente igual que los valores de Random Forest. La puntuación F1 para 'Physical' es del 0.95, y para 'Remote' es del 0.85. Esto indica un rendimiento dentro de todo equilibrado del modelo en ambas clases, pero con un mejor rendimiento para 'Physical'.

## Conclusión

Si bien los 3 modelos demuestran un rendimiento equilibrado, el Random Forest y el modelo SVM han demostrado ser más robustos en la clasificación de ambas clases. Con leves diferencias, ambos modelos son eficientes en su objetivo y la elección de uno sobre el otro dependerá de necesidades más específicas.